In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/colab_project')

!pip install -r requirements.txt
!pip install PyMuPDF python-pptx pandas openpyxl python-docx kaleido

import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
MODEL_PRICING = {
    "gemini-2.5-pro-preview-05-06": {
        "input_cost_per_1M_tokens": 1.25,
        "output_cost_per_1M_tokens": 10
    },
    "gemini-2.5-flash-preview-04-17": {
        "input_cost_per_1M_tokens": 0.15,
        "output_cost_per_1M_tokens": 0.6
    },
    "gemini-2.0-flash": {
        "input_cost_per_1M_tokens": 0.1,
        "output_cost_per_1M_tokens": 0.4
    },
    "gemini-2.0-flash-lite": {
        "input_cost_per_1M_tokens": 0.075,
        "output_cost_per_1M_tokens": 0.3
    }
}

agent_llm_list = [['sql_writer', 'sql_writer_generation'],
                  ['sql_validator', 'sql_validator'],
                  ['sql_judge', 'sql_writer_judge'],
                  ['bi_expert', 'bi_expert_generation'],
                  ['bi_judge', 'bi_expert_judge'],
                  ['python_writer', 'py_gen_generation'],
                  ['python_validator', 'py_fixer'],
                  ['python_judge', 'py_gen_judge'],
                  ['insights_generator', 'insights_generator'],
                  ['insights_judge', 'insights_judge']
                  ]

def calculate_llm_cost(final_state):
    total_cost = 0
    input_tokens = 0
    output_tokens = 0

    for agent_llm in agent_llm_list:
        llm_config = final_state[f'llm_config_{agent_llm[0]}']
        input_token_name = f'{agent_llm[1]}_input_tokens'
        output_token_name = f'{agent_llm[1]}_output_tokens'
        model_used = llm_config['model_name']
        input_cost = MODEL_PRICING[model_used]['input_cost_per_1M_tokens'] * final_state[
            input_token_name] / 1000000.0
        output_cost = MODEL_PRICING[model_used]['output_cost_per_1M_tokens'] * final_state[
            output_token_name] / 1000000.0
        total_cost += input_cost + output_cost
        input_tokens += final_state[input_token_name]
        output_tokens += final_state[output_token_name]

    return total_cost, input_tokens, output_tokens

# calculate_llm_cost(final_state)

In [ ]:
import workflow
import importlib # Import the importlib module
importlib.reload(workflow) # This reloads your main workflow.py

USER_STORIES_DOCX_PATH="/content/drive/My Drive/Datasets/User Stories/User Stories - March 2025_.docx"
MTSS_DOCS_FOLDER_PATH = "/content/drive/My Drive/Datasets/MTSS Documentation"
CORR_DATA_FILE1_PATH = "/content/drive/My Drive/Datasets/IAR_ELA.csv"
CORR_DATA_FILE2_PATH = "/content/drive/My Drive/Datasets/IAR_MATH.csv"

user_stories_text_content = workflow.read_docx(USER_STORIES_DOCX_PATH).strip()
mtss_docs_text_content = workflow.load_mtss_documentation_context(MTSS_DOCS_FOLDER_PATH).strip()
correlations_string_summary = workflow.full_correlation_analysis_string(CORR_DATA_FILE1_PATH, CORR_DATA_FILE2_PATH).strip()

Correlation datasets loaded and merged successfully.


In [ ]:
from ast import mod
import prompts
import workflow
import time
importlib.reload(prompts) # This reloads your main workflow.py
importlib.reload(workflow) # This reloads your main workflow.py


question = """What is the distribution of (Annual Grade) for students at each (Grade Level Current)?"""
model = "gemini-2.0-flash"
start_time = time.time()
final_state = workflow.run_workflow(question,
                                    user_stories_text=user_stories_text_content,
                                    mtss_docs_text=mtss_docs_text_content,
                                    correlations_text=correlations_string_summary,

                                    sql_writer_n_runs=3,
                                    bi_expert_n_runs=3,
                                    py_gen_n_runs=3,
                                    insights_n_runs=3,

                                    llm_sql_writer_model_name=model,
                                    llm_sql_validator_model_name=model,
                                    llm_sql_judge_model_name=model,
                                    llm_bi_expert_model_name=model,
                                    llm_bi_judge_model_name=model,
                                    llm_python_writer_model_name=model,
                                    llm_python_validator_model_name=model,
                                    llm_python_judge_model_name=model,
                                    llm_insights_generator_model_name=model,
                                    llm_insights_judge_model_name=model
                                    )
calculate_llm_cost(final_state)
print(time.time() - start_time)

Streaming output truncated to the last 5000 lines.
                               ' \xa0 management\t\r'
                               ' \xa0 issues),\t\r'
                               ' \xa0 and\t\r'
                               ' \xa0\n'
                               'learner\t\r'
                               ' \xa0 (e.g.,\t\r'
                               ' \xa0 skill\t\r'
                               ' \xa0 deficits)\t\r'
                               ' \xa0 for\t\r'
                               ' \xa0 why\t\r'
                               ' \xa0 the\t\r'
                               ' \xa0 student\t\r'
                               ' \xa0 is\t\r'
                               ' \xa0 not\t\r'
                               ' \xa0 meeting\t\r'
                               ' \xa0 expectations\t\r'
                               ' \xa0 and\t\r'
                               ' \xa0 collect\t\r'
                               ' \xa0\n'
                           

In [ ]:
if 'python_code_store_variables_dict' in final_state and 'df' in final_state['python_code_store_variables_dict']:
  print(final_state['python_code_store_variables_dict']['df_viz'])
else:
  print("DataFrame does not exist.")

KeyError: 'df_viz'

In [ ]:
if 'python_code_store_variables_dict' in final_state and 'fig' in final_state['python_code_store_variables_dict']:
  final_state['python_code_store_variables_dict']['fig'].show()
else:
  print("No visualization was generated.")

In [ ]:
final_state['generated_insights']

"Here's an analysis of the provided data, focusing on actionable insights and MTSS alignment:\n\n**Blurb:** The data highlights a correlation between current grade level and annual grades, with some homerooms showing stronger associations than others. Additionally, performance in English and Math are correlated.\n\n**Key Observations:**\n\n*   Current grade level strongly correlates with annual grades in both English and Math (0.802-0.842).\n*   Test scores in English and Math show a moderate positive correlation (0.611).\n*   Specific homerooms (e.g., I203, I204 in English and I210 in Math) show some correlation with annual grades (0.54-0.62), suggesting homeroom environment may influence performance.\n*   Students who partially meet expectations in one subject are likely to struggle in other subjects.\n\n**Suggestions (From the CORRELATION MATRIX SUMMARY):**\n\n*   Test Grades (Math) is correlated with Test Grades (English), ensuring a student does well in one will help them overall.

In [ ]:
from ast import mod
import importlib # Import the importlib module
import prompts
import workflow
import time
importlib.reload(prompts) # This reloads your main workflow.py
importlib.reload(workflow) # This reloads your main workflow.py

questions = ["""What is the distribution of (Annual Grade) for students at each (Grade Level Current)?""",
"""How does the (Test Grade) correlate with the (Test Scaled Score) across different (Subject Category)?""",
"""What is the average (Student Growth Percentile) for students in each (Current Homeroom), and are there significant differences between homerooms?"""]

questions = ["""What is the distribution of (Annual Grade) for students at each (Grade Level Current)?""",
"""How does the (Test Grade) correlate with the (Test Scaled Score) across different (Subject Category)?""",
"""What is the average (Student Growth Percentile) for students in each (Current Homeroom), and are there significant differences between homerooms?""",
"""Which (Tested School) has the highest average (Test Scaled Score), and how does this compare to the average (Test Scaled Score) of students at their (Current School)?""",
"""Are there any specific (Product) types that are associated with higher or lower (Test Scaled Score) results?""",
"""How does the (Test Primary Result Code) vary across different (Subject Category) and (Grade Level Current)?""",
"""What is the trend of (Test Scaled Score) over the (Year) for students in a specific (Grade Level Current)?""",
"""Can we identify students with a low (Annual Grade) but high (Student Growth Percentile), and what are their characteristics in terms of (Current Homeroom) and (Subject Category)?""",
"""What is the relationship between (Annual Grade) and (Test Grade) for students at different (Grade Level Current)?""",
"""How many students are associated with each (Product) and what is the average (Test Scaled Score) for each (Product)?""",
"""IAR Math""",
"""How does the (Test Scaled Score) correlate with the (Annual Grade) for students across different (Grade Level Current)?""",
"""What is the distribution of (Student Growth Percentile) within each (Subject Category), and are there any notable differences?""",
"""Are there specific (Tested School) or (Current School) locations where students tend to have higher or lower (Test Scaled Score) averages?""",
"""How does the (Product) used by students relate to their (Test Grade) and (Student Growth Percentile)?""",
"""What is the average (Test Scaled Score) for each (Grade Level Current), and how does this trend change over the (Year)?""",
"""Are there any patterns between (Current Homeroom) and (Test Primary Result Code) that might indicate specific teaching strategies or classroom environments that are more effective?""",
"""Which students, identified by (Student Name) and (Student ID), have shown the most significant (Student Growth Percentile) from one (Year) to the next?""",
"""How does the (Test Grade) compare to the (Annual Grade) for students, and are there any students who consistently perform differently on tests versus their overall grades?""",
"""What are the most frequent (Test Primary Result Code) values, and how do they vary across different (Subject Category)?""",
"""Can we identify any relationships between the (Current School) and the (Product) used by students, and does this have any impact on (Test Scaled Score)?"""]

potential_models = [
    #"gemini-2.0-flash-lite",
    "gemini-2.0-flash"
    #,"gemini-2.5-flash-preview-04-17"
    #,"gemini-2.5-pro-preview-05-06"
    ]


#questions = ["""What is the distribution of (Annual Grade) for students at each (Grade Level Current)?"""]
#potential_models = ["gemini-2.0-flash"]

model_dict = {}

for model in potential_models:
    total_cost = 0
    total_input_tokens = 0
    total_output_tokens = 0
    total_runs = 0
    total_run_time = 0
    if model == "gemini-2.5-pro-preview-05-06" and total_runs >= 5:
        break
    for question in questions:
        try:
            start_time = time.time()
            final_state = workflow.run_workflow(question,
                                    user_stories_text=user_stories_text_content,
                                    mtss_docs_text=mtss_docs_text_content,
                                    correlations_text=correlations_string_summary,

                                    sql_writer_n_runs=3,
                                    bi_expert_n_runs=3,
                                    py_gen_n_runs=3,
                                    insights_n_runs=3,

                                    llm_sql_writer_model_name=model,
                                    llm_sql_validator_model_name=model,
                                    llm_sql_judge_model_name=model,
                                    llm_bi_expert_model_name=model,
                                    llm_bi_judge_model_name=model,
                                    llm_python_writer_model_name=model,
                                    llm_python_validator_model_name=model,
                                    llm_python_judge_model_name=model,
                                    llm_insights_generator_model_name=model,
                                    llm_insights_judge_model_name=model)
            run_time = time.time() - start_time
            total_run_time += run_time
            run_cost, input_tokens, output_tokens = calculate_llm_cost(final_state)
            total_cost += run_cost
            total_input_tokens += input_tokens
            total_output_tokens += output_tokens
            total_runs += 1
        except Exception as e:
            print(f"An error occurred: {e}")
            continue
    model_dict[model] = [total_cost, total_input_tokens, total_output_tokens, total_runs, total_run_time]

for model in potential_models:
    print(f"Model: {model}")
    print(f"Total cost: {model_dict[model][0]}")
    print(f"Total input tokens: {model_dict[model][1]}")
    print(f"Total output tokens: {model_dict[model][2]}")
    print(f"Total runs: {model_dict[model][3]}")
    print(f"Average run time: {model_dict[model][4] / model_dict[model][3]}")
    print(f"Average cost per run: {model_dict[model][0] / model_dict[model][3]}")
    print(f"Average input tokens per run: {model_dict[model][1] / model_dict[model][3]}")
    print(f"Average output tokens per run: {model_dict[model][2] / model_dict[model][3]}")
    print("")

Streaming output truncated to the last 5000 lines.
                               ' \xa0 implementation\t\r'
                               ' \xa0 for\t\r'
                               ' \xa0 the\t\r'
                               ' \xa0 purpose\t\r'
                               ' \xa0 of\t\r'
                               ' \xa0\n'
                               'developing\t\r'
                               ' \xa0specific\t\r'
                               ' \xa0action\t\r'
                               ' \xa0plans\t\r'
                               ' \xa0to\t\r'
                               ' \xa0increase\t\r'
                               ' \xa0implementation\t\r'
                               ' \xa0levels.\t\r'
                               ' \xa0\n'
                               '22. Priority\t\r'
                               ' \xa0learning\t\r'
                               ' \xa0standards\t\r'
                               ' \xa0are\t\r'
                    